In [371]:
import pandas as pd
from math import log10
from random import randint

# 1- Escolha da database e separação das músicas

df_nirvana = pd.read_csv('Dados/nirvana_lyrics.csv')
df_pink_floyd = pd.read_csv('Dados\pink_floyd_lyrics.csv')

In [372]:
df_nirvana_treino = df_nirvana.iloc[:120]
df_pink_floyd_treino = df_pink_floyd.iloc[:120]

df_nirvana_teste = df_nirvana.iloc[120:140]
df_pink_floyd_teste = df_pink_floyd.iloc[120:140]

In [373]:
# 2- Probabilidade das músicas serem das bandas e total

probabilidade_nirvana = len(df_nirvana_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_pink_floyd = len(df_pink_floyd_treino) / (len(df_nirvana_treino) + len(df_pink_floyd_treino))
probabilidade_total = probabilidade_nirvana + probabilidade_pink_floyd

In [374]:
# 3- Exclusão de caracteres inúteis e repetições

import re
def arruma(lyrics):
    if isinstance(lyrics, str):
        lyrics = lyrics.lower()
        lyrics = re.sub(r"[.!@#$%&*(){}:;_?/<>+|=,\[\]\n']", '', lyrics)
        lyrics = re.sub(r'[\"]', '', lyrics)
        lyrics = re.sub(r"[’]", ' ', lyrics)

    return lyrics

def split_function(texto):
    return texto.split()

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].astype(str)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].astype(str)
df_nirvana_teste.loc[:,'lyrics'] = df_nirvana_teste['lyrics'].astype(str)
df_pink_floyd_teste.loc[:,'lyrics'] = df_pink_floyd_teste['lyrics'].astype(str)

df_nirvana_treino.loc[:,'lyrics'] = df_nirvana_treino['lyrics'].apply(arruma)
df_pink_floyd_treino.loc[:,'lyrics'] = df_pink_floyd_treino['lyrics'].apply(arruma)
df_nirvana_teste.loc[:,'lyrics'] = df_nirvana_teste['lyrics'].apply(arruma)
df_pink_floyd_teste.loc[:,'lyrics'] = df_pink_floyd_teste['lyrics'].apply(arruma)

df_nirvana_treino = pd.DataFrame(df_nirvana_treino)
df_pink_floyd_treino = pd.DataFrame(df_pink_floyd_treino)
df_nirvana_teste = pd.DataFrame(df_nirvana_teste)
df_pink_floyd_teste = pd.DataFrame(df_pink_floyd_teste)

def keep_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return ' '.join(unique_words)

df_nirvana_treino['lyrics'] = df_nirvana_treino['lyrics'].apply(keep_unique_words)
df_pink_floyd_treino['lyrics'] = df_pink_floyd_treino['lyrics'].apply(keep_unique_words)
df_nirvana_teste['lyrics'] = df_nirvana_teste['lyrics'].apply(keep_unique_words)
df_pink_floyd_teste['lyrics'] = df_pink_floyd_teste['lyrics'].apply(keep_unique_words)

df_total_treino = pd.concat([df_nirvana_treino, df_pink_floyd_treino], ignore_index=True)
df_total_teste = pd.concat([df_nirvana_teste, df_pink_floyd_teste], ignore_index=True)


In [375]:
# 4- Separação das músicas por palavras

palavras_por_musica_nirvana = {}
palavras_por_musica_pink_floyd = {}
palavras_por_musica_total = {}

contador1 = 0
contador2 = 0
contador3 = 0

for musicas_n in df_nirvana_treino['lyrics']:
    palavras_por_musica_nirvana[contador1] = musicas_n.split()
    contador1 += 1

for musicas_pf in df_pink_floyd_treino['lyrics']:
    palavras_por_musica_pink_floyd[contador2] = musicas_pf.split()
    contador2 += 1

for musicas_total in df_total_treino['lyrics']:
    palavras_por_musica_total[contador3] = musicas_total.split()
    contador3 += 1

In [376]:
# 5- Frequência da palavra dado música

frequencia_nirvana = {}
frequencia_pink_floyd = {}
frequencia_total = {}

for musicas in palavras_por_musica_nirvana.values():
    for palavras in musicas:
        if palavras in frequencia_nirvana:
            frequencia_nirvana[palavras] += 1
        else:
            frequencia_nirvana[palavras] = 1

for musicas in palavras_por_musica_pink_floyd.values():
    for palavras in musicas:
        if palavras in frequencia_pink_floyd:
            frequencia_pink_floyd[palavras] += 1
        else:
            frequencia_pink_floyd[palavras] = 1

for musicas in palavras_por_musica_total.values():
    for palavras in musicas:
        if palavras in frequencia_total:
            frequencia_total[palavras] += 1
        else:
            frequencia_total[palavras] = 1

In [377]:
# 6- Probabilidade de cada palavra por banda e geral

probabilidade_por_palavra_nirvana = {}
probabilidade_por_palavra_pink_floyd = {}
probabilidade_por_palavra_total = {}

for palavra,quantidade in frequencia_nirvana.items():
    probabilidade_por_palavra_nirvana[palavra] = quantidade/len(palavras_por_musica_nirvana)

for palavra,quantidade in frequencia_pink_floyd.items():
    probabilidade_por_palavra_pink_floyd[palavra] = quantidade/len(palavras_por_musica_pink_floyd)

for palavra,quantidade in frequencia_total.items():
    probabilidade_por_palavra_total[palavra] = quantidade/len(palavras_por_musica_total)

In [378]:
# 7- Teorema de Bayes e Precisão

def teorema_de_bayes(musica):
    musica = musica.split()
    propabilidade_final_pf = 0
    propabilidade_final_n = 0

    for palavras in musica:
        if palavras in probabilidade_por_palavra_pink_floyd:
            soma_pf = log10(probabilidade_por_palavra_pink_floyd[palavras])
            subtracao_pf = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_pf += soma_pf
            propabilidade_final_pf -= subtracao_pf
    propabilidade_final_pf *= probabilidade_pink_floyd
        

    for palavras in musica:
        if palavras in probabilidade_por_palavra_nirvana:
            soma_n = log10(probabilidade_por_palavra_nirvana[palavras])
            subtracao_n = log10(probabilidade_por_palavra_total[palavras])
            propabilidade_final_n += soma_n
            propabilidade_final_n -= subtracao_n
    propabilidade_final_n *= probabilidade_nirvana
    
    return propabilidade_final_pf, propabilidade_final_n


In [383]:
acu_nirvana = 0
acu_pink_floyd = 0

not_acu_nirvana = 0
not_acu_pink_floyd = 0

for i in range(18):
    index = randint(120,139)
    musica1 = df_nirvana_teste['lyrics'][index]
    final_pf, final_n = teorema_de_bayes(musica1)
    if final_n > final_pf:
        acu_nirvana += 1

for i in range(18):
    index = randint(120,139)
    musica2 = df_pink_floyd_teste['lyrics'][index]
    final_pf, final_n = teorema_de_bayes(musica2)
    if final_pf > final_n:
        acu_pink_floyd += 1
        
acu_total = ((acu_nirvana + acu_pink_floyd) * 100)/36

print(f"A precisão é de {acu_total:.2f}%")

A precisão é de 91.67%
